# Blue Mountains Analysis

## Prepatory work

In [4]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format='retina'


from __future__ import absolute_import, division, print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import GridSpec
import seaborn as sns
import mpld3
import numpy as np
import pandas as pd
import os, sys
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
sns.set_context("poster", font_scale=1.3)

from IPython.display import Image


#import hdbscan
from sklearn.datasets import make_blobs
import time

#import bokeh
from bokeh.plotting import figure, show, output_notebook


In [5]:
#bring in table containing initial values (cond table)
initial = pd.read_excel(open('data/initial.xlsx', 'rb'), sheetname='combo')

In [6]:
#show first 5 lines of dataframe
initial.to_csv('outputs/initial.csv')
initial.head(n=5)

,biosum_cond_id,biosum_plot_id,invyr,condid,condprop,landclcd,fortypcd,ground_land_class_pnw,owncd,owngrpcd,...,fvs_filename,idb_cond_id,idb_plot_id,cn,biosum_status_cd,Hazard,Surface_Flame_Height,PTorch,TorchIndex,MorVolPer
0,1200741010706100863620001,120074101070610086362000,2007,1,1.017708,1,321,120,11,10,...,41863621.fvs,NaN,NaN,171243552020004,1,2,3.430332,0.364541,29.663570,40.428998
1,1200741010706100873630001,120074101070610087363000,2007,1,1.017708,1,201,120,11,10,...,41873631.fvs,NaN,NaN,171243758020004,1,3,3.815775,0.999502,18.385487,84.297243
2,1200741010706100892580001,120074101070610089258000,2007,1,1.017708,1,321,120,11,10,...,41892581.fvs,NaN,NaN,171243740020004,1,1,1.906024,0.000000,210.137131,99.987517
3,1200741010706100968720001,120074101070610096872000,2007,1,0.915437,1,221,120,11,10,...,41968721.fvs,NaN,NaN,171243766020004,1,4,4.744007,0.912882,0.000000,90.182055
4,1200741010706300564370001,120074101070630056437000,2007,1,0.618935,1,201,120,11,10,...,41564371.fvs,NaN,NaN,171243948020004,1,1,5.206946,0.156523,107.447495,20.555877


# How many forested acres are currently at high fire risk (by landowner class)?

In [7]:
#group by owner and hazard score, display acres
groupby_object = initial[['owngrpcd', 'Hazard', 'acres']].groupby(['owngrpcd', 'Hazard'])

In [8]:
#calculate the sum of acres, by ownergroup and hazard score, for intial conditions, also provide a count. 
initial_sum = groupby_object.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .join(pd.DataFrame(groupby_object.size(), 
                                      columns=['counts']))\
                .reset_index()
                
print(initial_sum)

   owngrpcd  Hazard      acres sum  counts
0        10       0  349403.040186     214
1        10       1  580876.140878     356
2        10       2  546552.250725     329
3        10       3  407742.751668     234
4        10       4  269700.550173     147
5        40       0  103009.316626      19
6        40       1  248283.002003      46
7        40       2  242613.253318      40
8        40       3  135544.030100      23
9        40       4  104257.392476      19


Above table: by owner group, the number and percentage of acres in for each Hazard score. 

Notes: Similiar percentage of acres in each class between owner groups, surprisingly. Would have suspected that federal lands would have a higher percentage of acres in the higher hazard score groups. 

In [9]:
#sum up acres by owner group
fed=initial_sum[initial_sum["owngrpcd"]==10].sum()["acres sum"]
private=initial_sum[initial_sum["owngrpcd"]==40].sum()["acres sum"]
total_acres=fed + private

print(fed)
print(private)
print(total_acres)

2154274.73363
833706.994523
2987981.72815


Acre totals (no filtering) by owner group:
* **Fed:** 2,154,274
* **Private:** 833,706
* **Total acres:** 2,987,981

In [10]:
#find percentage of acres in each hazard class by owner group
initial_sum['percent']=np.NaN
initial_sum['percent'][initial_sum.owngrpcd==10]=(initial_sum['acres sum']/fed)*100
initial_sum['percent'][initial_sum.owngrpcd==40]=(initial_sum['acres sum']/private)*100
print(initial_sum)

   owngrpcd  Hazard      acres sum  counts    percent
0        10       0  349403.040186     214  16.219057
1        10       1  580876.140878     356  26.963884
2        10       2  546552.250725     329  25.370592
3        10       3  407742.751668     234  18.927147
4        10       4  269700.550173     147  12.519320
5        40       0  103009.316626      19  12.355578
6        40       1  248283.002003      46  29.780607
7        40       2  242613.253318      40  29.100542
8        40       3  135544.030100      23  16.257994
9        40       4  104257.392476      19  12.505280


In [11]:
#export to graph in R - see graphs created below
initial.to_csv('outputs/initial.csv')

initial_sum.to_csv('outputs/initial_sum.csv')


<img src="graphs/initialHazard_byOwnerGroup.jpg",width=500,height=600>

<img src="graphs/initialHazard_byOwnerGroup_p.jpg",width=500,height=600>

![](.graphs/initialHazard_byOwnerGroup.png =100x20)

# What was the change in effectiveness after 'best' treatment?

This SQL query was used to create the table "weighted_no_NR_opt" based on the core analysis scenario "weighted_nNR"

``` mysql
SELECT cycle1_best_rx_summary.biosum_cond_id, cycle1_best_rx_summary.optimization_value, cycle1_best_rx_summary.acres, cycle1_best_rx_summary.owngrpcd, cycle1_effective.pre_variable1_value AS preHazard
FROM cycle1_effective INNER JOIN (cycle1_best_rx_summary INNER JOIN cycle1_optimization ON (cycle1_best_rx_summary.biosum_cond_id = cycle1_optimization.biosum_cond_id) AND (cycle1_best_rx_summary.rx = cycle1_optimization.rx)) ON (cycle1_effective.rx = cycle1_best_rx_summary.rx) AND (cycle1_effective.biosum_cond_id = cycle1_best_rx_summary.biosum_cond_id)
WHERE (((cycle1_effective.rxcycle)="1"));

```

In [12]:
#read in the cycle1_optimization created by SQL above from tables in core analysis
w_nNR_change = pd.read_excel(open('data/weighted_no_NR_opt.xlsx', 'rb'), sheetname='opt', lowmemory=False)

w_nNR_change.biosum_cond_id = w_nNR_change.biosum_cond_id.astype(str)
w_nNR_change.head(n=3)

,biosum_cond_id,optimization_value,acres,owngrpcd,preHazard,rxpackage
0,1200741010700100571330001,-1.475,1998.090515,10,2.525,3
1,1200741010700100600620001,-0.225,1820.489084,10,1.750,34
2,1200741010700100714590001,-0.925,1711.995996,10,4.000,27


In [13]:
#create a bin column and assign based on change in optimiation value
w_nNR_change['bin'] = pd.cut(w_nNR_change['optimization_value'], [-4, -3, -2, -1, 0])
#create a pre_bin column and assign based on initial hazard score
w_nNR_change['pre_bin'] = pd.cut(w_nNR_change['preHazard'], [0,1,2,3,4])

w_nNR_change.head(n=5)

,biosum_cond_id,optimization_value,acres,owngrpcd,preHazard,rxpackage,bin,pre_bin
0,1200741010700100571330001,-1.475,1998.090515,10,2.525,3,"(-2, -1]","(2, 3]"
1,1200741010700100600620001,-0.225,1820.489084,10,1.750,34,"(-1, 0]","(1, 2]"
2,1200741010700100714590001,-0.925,1711.995996,10,4.000,27,"(-1, 0]","(3, 4]"
3,1200741010700100717720001,-0.225,1711.995996,10,2.775,25,"(-1, 0]","(2, 3]"
4,1200741010700100736160001,-0.025,2001.387821,10,1.000,6,"(-1, 0]","(0, 1]"


In [14]:
#group by pre_bin, bin, and ownergroup and sum the number of acres
w_nNR_change.groupby(['pre_bin', 'bin', 'owngrpcd'], as_index=False)['acres'].sum()

#this table was used to create the change in hazard score table 
w_nNR_change.head(n=2)

,biosum_cond_id,optimization_value,acres,owngrpcd,preHazard,rxpackage,bin,pre_bin
0,1200741010700100571330001,-1.475,1998.090515,10,2.525,3,"(-2, -1]","(2, 3]"
1,1200741010700100600620001,-0.225,1820.489084,10,1.750,34,"(-1, 0]","(1, 2]"


#  What is treatment popularity [acres picked for each treatment] (when cost is no object, and only seeking to maximize the multi-point score described below, or the multi-point score difference from grow only)?

In [15]:
#read in the compile_Effective table (which is the cycle1_best_rx_summary from CORE ANALYSIS scenario weighted_nNR)
w_nNR = pd.read_excel(open('data/weighted_noNR.xlsx', 'rb'), sheetname='cycle1_best_rx_summary', lowmemory=False)


In [16]:
#display top 5 lines of table
w_nNR.head(n=5)

,biosum_cond_id,rx,acres,owngrpcd,optimization_value,tiebreaker_value,rx_intensity
0,1200741010700100571330001,102,1998.090515,10,-1.475,84.491959,NaN
1,1200741010700100600620001,305,1820.489084,10,-0.225,83.301268,NaN
2,1200741010700100714590001,208,1711.995996,10,-0.925,67.847637,NaN
3,1200741010700100717720001,206,1711.995996,10,-0.225,79.823242,NaN
4,1200741010700100736160001,115,2001.387821,10,-0.025,23.657479,NaN


In [17]:
#group by owner and hazard score, display acres
groupby_w = w_nNR[['owngrpcd', 'rx', 'acres']].groupby(['owngrpcd', 'rx'])


In [18]:
#calculate the sum of acres, by ownergroup and hazard score, for intial conditions, also provide a count. 
w_nNR_sum = groupby_w.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .join(pd.DataFrame(groupby_w.size(), 
                                      columns=['counts']))\
                .reset_index()

#print the first five lines of the new table
w_nNR_sum.head(n=5)

,owngrpcd,rx,acres sum,counts
0,10,100,31202.900491,17
1,10,101,7755.767716,5
2,10,102,13252.221828,8
3,10,103,5867.229996,4
4,10,104,79343.566174,44


In [19]:
#find percentage of acres treated by each rx by owner group
#NOTE: percentages are of owner group acre totals, NOT of combined total acres
w_nNR_sum['percent']=np.NaN
w_nNR_sum['percent'][w_nNR_sum.owngrpcd==10]=(w_nNR_sum['acres sum']/fed)*100
w_nNR_sum['percent'][w_nNR_sum.owngrpcd==40]=(w_nNR_sum['acres sum']/private)*100
w_nNR_sum.head(n=5)

,owngrpcd,rx,acres sum,counts,percent
0,10,100,31202.900491,17,1.448418
1,10,101,7755.767716,5,0.360018
2,10,102,13252.221828,8,0.615159
3,10,103,5867.229996,4,0.272353
4,10,104,79343.566174,44,3.683076


Dataframe **w_nNR_sum**: ownergroup, rx, sum of acres, counts, and percent of subgroup total acres, that COULD be treated effectively by at least one treatment. 

In [20]:
#total up acres that are able to be treated by an rx by ownertype
fed_treat=w_nNR_sum[w_nNR_sum["owngrpcd"]==10].sum()["acres sum"]
private_treat=w_nNR_sum[w_nNR_sum["owngrpcd"]==40].sum()["acres sum"]

In [21]:
print(fed_treat)
print(private_treat)

1236856.41436
396177.719668


Total acres that could be treated by any rx:
* **Federal**: 1,236,856
* **Private**: 396,178

In [22]:
#total up percent of acres that are able to be treated by an rx by ownertype
fed_treatP=w_nNR_sum[w_nNR_sum["owngrpcd"]==10].sum()["percent"]
private_treatP=w_nNR_sum[w_nNR_sum["owngrpcd"]==40].sum()["percent"]

In [23]:
print(fed_treatP)
print(private_treatP)

57.4140519336
47.5200186961


**57% of federal acres** could be effectively treated by one or more of the treatment packages. 

**48% of private acres** coule be effectively treated by one or more of the treatment packages.  

This is not considering net revenue.  

In [24]:
#calculate fed acres not treated
fed_notTreated=fed-fed_treat

#calculate private acres not treated
private_notTreated=private-private_treat

#calculate fed acres not treated PERCENT
fed_notTreatedP=100-fed_treatP

#calculate private acres not treated PERCENT
private_notTreatedP=100-private_treatP

#append to fed and private NON TREATED acres to the dataframe
#fedrow = [10, "000", fed_notTreated, 0, fed_notTreatedP]

#w_nNR_sum.loc[len(w_nNR_sum)] = fedrow

# print values
print(fed_notTreated)
print(fed_notTreatedP)
print(private_notTreated)
print(private_notTreatedP)

917418.319272
42.5859480664
437529.274855
52.4799813039


Total acres that could **NOT** be treated by any rx:
* **Federal**: 917,418 (43%)
* **Private**: 437,529 (52%)

Notes: Quite a large majority.  Remember, this is looking at the weighted average over 40 years.  
To do: Look into how different this looks when only considering First Entry Hazard Scores. 

## Quick aside, how many acres can be effectively treated using the only First Entry pre/post values, rather than weighted average vs. grow only.  

In [25]:
#read in the compile_Effective_FE table (which is the cycle1_best_rx_summary from CORE ANALYSIS First Entry scenario, not based on NET REV)
fe_nNR = pd.read_excel(open('data/compile_Effective_FE.xlsx', 'rb'), sheetname='cycle1_best_rx_summary', lowmemory=False)


In [26]:
#display top 5 lines of table
fe_nNR.head(n=5)

,biosum_cond_id,rx,acres,owngrpcd,optimization_value,tiebreaker_value,rx_intensity
0,1200741010700100571330001,105,1998.090515,10,-3,9.604044,NaN
1,1200741010700100600620001,304,1820.489084,10,-2,11.027757,NaN
2,1200741010700100639370001,211,2002.540123,10,-1,11.642599,NaN
3,1200741010700100712680001,107,2099.998873,10,-1,13.215190,NaN
4,1200741010700100714590001,208,1711.995996,10,-4,26.586103,NaN


In [27]:
#group by owner and hazard score, display acres
groupby_w = fe_nNR[['owngrpcd', 'rx', 'acres']].groupby(['owngrpcd', 'rx'])


In [28]:
#calculate the sum of acres, by ownergroup and hazard score, for intial conditions, also provide a count. 
fe_nNR = groupby_w.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .join(pd.DataFrame(groupby_w.size(), 
                                      columns=['counts']))\
                .reset_index()

#print the first five lines of the new table
fe_nNR.head(n=5)

,owngrpcd,rx,acres sum,counts
0,10,100,11673.975677,9
1,10,101,5974.139375,4
2,10,102,8918.095113,6
3,10,103,13506.207104,7
4,10,104,77155.435738,42


In [29]:
#find percentage of acres treated by each rx by owner group
#NOTE: percentages are of owner group acre totals, NOT of combined total acres
fe_nNR['percent']=np.NaN
fe_nNR['percent'][fe_nNR.owngrpcd==10]=(fe_nNR['acres sum']/fed)*100
fe_nNR['percent'][fe_nNR.owngrpcd==40]=(fe_nNR['acres sum']/private)*100
fe_nNR.head(n=5)

,owngrpcd,rx,acres sum,counts,percent
0,10,100,11673.975677,9,0.541898
1,10,101,5974.139375,4,0.277316
2,10,102,8918.095113,6,0.413972
3,10,103,13506.207104,7,0.626949
4,10,104,77155.435738,42,3.581504


Dataframe **fe_nNR**: ownergroup, rx, sum of acres, counts, and percent of subgroup total acres, that COULD be treated effectively by at least one treatment. 

In [30]:
#total up acres that are able to be treated by an rx by ownertype
fed_treat2=fe_nNR[fe_nNR["owngrpcd"]==10].sum()["acres sum"]
private_treat2=fe_nNR[fe_nNR["owngrpcd"]==40].sum()["acres sum"]

In [31]:
print(fed_treat2)
print(private_treat2)

1505322.99661
486313.19848


Total acres that could be treated by any rx in First Entry scenario:
* **Federal**: 1,505,323
* **Private**: 486,313

In [32]:
#total up percent of acres that are able to be treated by an rx by ownertype
fed_treat2P=fe_nNR[fe_nNR["owngrpcd"]==10].sum()["percent"]
private_treat2P=fe_nNR[fe_nNR["owngrpcd"]==40].sum()["percent"]

In [33]:
print(fed_treat2P)
print(private_treat2P)

69.8760920839
58.3314283885


**70% of federal acres** could be effectively treated by one or more of the treatment packages. 

**58% of private acres** coule be effectively treated by one or more of the treatment packages.  


In [34]:
#what is the treatment popularity when NR is not considered
#export to csv to graph in R
w_nNR_sum.to_csv("outputs/w_nNR_sum.csv")

<img src="treatPopAcres.png" style="width: 1000px;"/>

#  What is treatment popularity when NR must be positive? 

In [75]:
#read in the compile_Effective when NR must be greater than 0.
w_NR = pd.read_excel(open('data/weighted_NR.xlsx', 'rb'), sheetname='cycle1_best_rx_summary', lowmemory=False)

In [76]:
#display top 5 lines of table
w_NR.head(n=5)

,biosum_cond_id,rx,acres,owngrpcd,optimization_value,tiebreaker_value,rx_intensity
0,1200741010700100571330001,106,1998.090515,10,-0.550,84.547150,NaN
1,1200741010700100600620001,305,1820.489084,10,-0.225,83.301268,NaN
2,1200741010700100736160001,115,2001.387821,10,-0.025,23.657479,NaN
3,1200741010700100745730001,114,1764.144741,10,-0.225,61.144567,NaN
4,1200741010700100814910001,116,1799.216540,10,-0.450,85.919490,NaN


In [77]:
#group by owner and hazard score, display acres
groupby_wNR = w_NR[['owngrpcd', 'rx', 'acres']].groupby(['owngrpcd', 'rx'])

In [78]:
#calculate the sum of acres, by ownergroup and hazard score, for intial conditions, also provide a coont. 
w_NR_sum = groupby_wNR.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .join(pd.DataFrame(groupby_wNR.size(), 
                                      columns=['counts']))\
                .reset_index()
                
w_NR_sum.head(n=5)

,owngrpcd,rx,acres sum,counts
0,10,100,25577.269730,15
1,10,101,1447.743410,1
2,10,102,12830.591475,8
3,10,103,1885.575859,1
4,10,104,121827.596074,66


In [79]:
#find percentage of acres treated by each rx by owner group
w_NR_sum['percent']=np.NaN
w_NR_sum['percent'][w_NR_sum.owngrpcd==10]=(w_NR_sum['acres sum']/fed)*100
w_NR_sum['percent'][w_NR_sum.owngrpcd==40]=(w_NR_sum['acres sum']/private)*100
w_NR_sum.head(n=5)

,owngrpcd,rx,acres sum,counts,percent
0,10,100,25577.269730,15,1.187280
1,10,101,1447.743410,1,0.067203
2,10,102,12830.591475,8,0.595588
3,10,103,1885.575859,1,0.087527
4,10,104,121827.596074,66,5.655156


In [80]:
#total up acres that are able to be treated by an rx by ownertype (with net rev > 0)
fed_treatNR=w_NR_sum[w_NR_sum["owngrpcd"]==10].sum()["acres sum"]
private_treatNR=w_NR_sum[w_NR_sum["owngrpcd"]==40].sum()["acres sum"]

In [81]:
print(fed_treatNR)
print(private_treatNR)

1021385.08207
323837.051562


In [82]:
#total up percent of acres that are able to be treated by an rx by ownertype (with net rev > 0)
fed_treatPnr=w_NR_sum[w_NR_sum["owngrpcd"]==10].sum()["percent"]
private_treatPnr=w_NR_sum[w_NR_sum["owngrpcd"]==40].sum()["percent"]
w_NR_sum.head(n=3)

,owngrpcd,rx,acres sum,counts,percent
0,10,100,25577.269730,15,1.187280
1,10,101,1447.743410,1,0.067203
2,10,102,12830.591475,8,0.595588


In [83]:
print(fed_treatPnr)
print(private_treatPnr)

47.4120160314
38.8430292284


In [84]:
#export to csv to graph in R
w_NR_sum.to_csv("outputs/w_NR_sum.csv")

<img src="graphs/treatPopAcres.png" width="500", height="250"/>

When NET REV must be greater than 0, the number of acres that can be effectively treated drops. 
* **Federal**: 1,021,385 (47%) ((down from 56%))
* **Private**: 323,837 (38%) ((down from 47%))

# What does the hazard look like over time when based on FE_Hazard? 

In [85]:
#read in the compile_Effective_FE table (which is the cycle1_best_rx_summary from CORE ANALYSIS First Entry scenario,
#not based on NET REV)
#with all years included"
fe_nNR_year = pd.read_excel(open('data/compile_Effective_FE_year2.xlsx', 'rb'), sheetname='combine_effective', lowmemory=False)

In [86]:
fe_nNR_year.head(n=3)

,biosum_cond_id,rx,rxcycle,Year,Hazard,owngrpcd,acres
0,1200841010804900854490001,116,1,1,2,10,1818.842028
1,1201341020302300867220001,116,1,1,0,10,1254.431692
2,1200941010900100788610001,116,1,1,2,10,1620.599417


In [87]:
#group by owner and hazard score and year, display acres
groupby_a = fe_nNR_year[['acres', 'Year', 'Hazard']].groupby(['Year', 'Hazard'])

In [88]:
#calculate the sum of acres, by ownergroup and hazard score and Year, also provide a count. 
fe_nNR_year = groupby_a.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .reset_index()

#print the first five lines of the new table
fe_nNR_year.head(n=5)

,Year,Hazard,acres sum
0,1,0,106551.455881
1,1,1,514932.791554
2,1,2,650817.195540
3,1,3,436049.573924
4,1,4,283285.178193


In [89]:
acresFEtreat = fed_treat2+private_treat2
print(acresFEtreat)

1991636.19509


In [92]:
#find percentage of acres treated by each rx by owner group
#NOTE: percentages are of owner group acre totals, NOT of combined total acres
fe_nNR_year['percent']=np.NaN
fe_nNR_year['percent']=(fe_nNR_year['acres sum']/acresFEtreat)*100

#output to csv file for plotting in R.  
fe_nNR_year.to_csv("outputs/fe_nNR_year.csv")

In [36]:
Image(url= "graphs/feHazard.png", width=700, height=350)

# What does hazard look like over time when effectiveness is based on cycle 1 only? 

In [39]:
#read in the compile_Effective_FE table (which is the cycle1_best_rx_summary from CORE ANALYSIS cycle 1 scenario,
#not based on NET REV)
#with all years included"
c1_nNR_year = pd.read_excel(open('data/compile_Effective_cycle1_year.xlsx', 'rb'), sheetname='combine_effective_cycle1', lowmemory=False)

In [40]:
c1_nNR_year.head(n=3)

,biosum_cond_id,rx,rxcycle,Year,Hazard,owngrpcd,acres
0,1200841010804900854490001,114,1,1,2,10,1818.842028
1,1200741010702300745190001,108,1,1,1,10,1171.040358
2,1201441020405900515690002,108,1,1,2,10,946.265915


In [41]:
#group by owner and hazard score and year, display acres
groupby_c = c1_nNR_year[['acres', 'Year', 'Hazard']].groupby(['Year', 'Hazard'])


In [42]:
#calculate the sum of acres, by ownergroup and hazard score and Year, also provide a count. 
c1_nNR_year = groupby_c.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .reset_index()

#print the first five lines of the new table
c1_nNR_year.head(n=5)

,Year,Hazard,acres sum
0,1,1,308305.663579
1,1,2,351953.414571
2,1,3,160151.637342
3,1,4,64626.289142
4,3,0,485397.710030


In [43]:
tots=308305.7+351953.4+160151.6+64626.3

In [44]:
#find percentage of acres treated by each rx by owner group
#NOTE: percentages are of owner group acre totals, NOT of combined total acres
c1_nNR_year['percent']=np.NaN
c1_nNR_year['percent']=(c1_nNR_year['acres sum']/tots)*100
c1_nNR_year.to_csv("outputs/c1_nNR_year.csv")

In [45]:
Image(url= "graphs/cy1Hazard.png"S, width=500, height=250)


# For weighted average treatments that must pay for themselves, what is the acres, wood produced, and net revenue for 20 year increments?


Use query below in the core scenrio db for weighted_NR to create the effective_NR_outputs table

``` mysql
SELECT cycle1_best_rx_summary.biosum_cond_id, cycle1_best_rx_summary.rx, cycle1_best_rx_summary.rxpackage, cycle1_best_rx_summary.acres, cycle1_best_rx_summary.owngrpcd, product_yields_net_rev_costs_summary_by_rx.chip_yield_cf, product_yields_net_rev_costs_summary_by_rx.merch_yield_cf, product_yields_net_rev_costs_summary_by_rx.max_nr_dpa, product_yields_net_rev_costs_summary_by_rx.rxcycle
FROM product_yields_net_rev_costs_summary_by_rx INNER JOIN cycle1_best_rx_summary ON (product_yields_net_rev_costs_summary_by_rx.rx = cycle1_best_rx_summary.rx) AND (product_yields_net_rev_costs_summary_by_rx.biosum_cond_id = cycle1_best_rx_summary.biosum_cond_id);

```

In [53]:
#read in the table created by SQL above from tables in weightedAvg_NR core analysis
w_NR_output = pd.read_excel(open('data/effective_NR_outputs.xlsx', 'rb'), sheetname='outputs', lowmemory=False)

In [54]:
#create column period 
w_NR_output.period = w_NR_output.ix[w_NR_output.rxcycle.isin([1, 2]), 'period'] = 1
w_NR_output.period = w_NR_output.ix[w_NR_output.rxcycle.isin([3, 4]), 'period'] = 2

In [55]:
#create an expansion column = acres * max_nr_dpa
w_NR_output['expansion'] = w_NR_output['acres'] * w_NR_output['max_nr_dpa']  # assigned to a column

In [56]:
#expand merch and chip values
w_NR_output['merch_ex'] = w_NR_output['acres'] * w_NR_output['merch_yield_cf']
w_NR_output['chip_ex'] = w_NR_output['acres'] * w_NR_output['chip_yield_cf']

In [57]:
w_NR_output.head(n=3)

,biosum_cond_id,rx,rxpackage,acres,owngrpcd,chip_yield_cf,merch_yield_cf,max_nr_dpa,rxcycle,period,expansion,merch_ex,chip_ex
0,1201241020200100570760001,208,27,1820.5,10,1967.2,4551.0,6452.7,1,1.0,11747147.7,8285097.1,3581269.8
1,1201241020202300639610001,208,27,1229.5,10,1364.2,2132.9,1273.8,1,1.0,1566153.4,2622403.7,1677283.6
2,1201241020204900907840001,208,27,903.4,10,1296.2,1006.7,-675.3,1,1.0,-610106.4,909456.4,1171005.6


In [58]:
#group by owngrpcd, period
groupby_o = w_NR_output[['owngrpcd','period','acres', 'chip_ex', 'merch_ex', 'expansion']]\
                .groupby(['owngrpcd','period'])

In [59]:
#calculate the sum of acre for minDBH groups, rxpackages and provide count of stands.
pd.set_option('display.float_format', lambda x: '%.1f' % x)
w_NR_outputAgg = groupby_o.agg('sum')\
                .rename(columns = lambda x: x + '_sum')\
                .reset_index()
            

w_NR_outputAgg.head(n=20)

,owngrpcd,period,acres_sum,chip_ex_sum,merch_ex_sum,expansion_sum
0,10,1.0,660031.9,670111022.5,1476197458.5,1405605143.6
1,10,2.0,723855.2,505641692.9,899839807.3,233685184.5
2,40,1.0,128784.6,146984158.4,295171331.1,306976317.4
3,40,2.0,284832.4,203554321.0,399767823.9,195010004.4


# How many fewer acres can be effectively treated if DBH cap is 21 rather than 30? How many fewer acres can be treated if this is imposed and each acre must pay for itself? 

Create a table using below query in the Core Analysis scenario weighted_nNR

``` mysql
SELECT cycle1_optimization.biosum_cond_id, cycle1_optimization.rxpackage, cycle1_optimization.rx, cycle1_optimization.change_value AS ptorchChange, tiebreaker.post_variable1_value AS TIPost, cond.acres
FROM (cond INNER JOIN tiebreaker ON cond.biosum_cond_id = tiebreaker.biosum_cond_id) INNER JOIN cycle1_optimization ON (cycle1_optimization.rxcycle = tiebreaker.rxcycle) AND (cycle1_optimization.rxpackage = tiebreaker.rxpackage) AND (tiebreaker.biosum_cond_id = cycle1_optimization.biosum_cond_id);

```

In [60]:
#read in the cycle1_optimization created by SQL above from tables in core analysis
w_nNR_opt = pd.read_excel(open('data/weighted_noNR_opt2.xlsx', 'rb'), sheetname='export', lowmemory=False)

w_nNR_opt.biosum_cond_id = w_nNR_opt.biosum_cond_id.astype(str)
w_nNR_opt.rx = w_nNR_opt.rx.astype(str)
w_nNR_opt.rxpackage = w_nNR_opt.rxpackage.astype(str)

In [61]:
# list of rx values that coorespond to 21 and 30 inch DBH caps
cap21 = ('100', '101', '102', '103', '200', '201', '202', '203')

cap30 = ('104', '105', '106', '107', '204', '205', '206', '207')

In [62]:
#filter optimiztion table by cap21 
w_nNR_21=w_nNR_opt[w_nNR_opt['rx'].isin(cap21)]

print(w_nNR_21.shape)
w_nNR_21.head(n=5)


(2906, 6)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres
0,1200741010700100571330001,1,100,-1.3,70.5,1998.1
1,1200741010700100714590001,1,100,-0.5,76.8,1712.0
2,1200741010700100814910001,1,100,-0.3,88.9,1799.2
3,1200741010700100880590002,1,100,-0.4,47.1,652.2
4,1200741010702300724110001,1,100,-1.4,56.1,1997.9


In [63]:
#Optimize by stand, max reduction in PTorch, create new table w_nNR_21opt

from pandasql import *

pysqldf = lambda q: sqldf(q, globals())

q  = """
SELECT *, MIN(wHazardChange) FROM w_nNR_21 WHERE wHazardChange < 0 GROUP BY biosum_cond_id;
"""
w_nNR_21opt = pysqldf(q)

print(w_nNR_21opt.shape)
w_nNR_21opt.biosum_cond_id = w_nNR_21opt.biosum_cond_id.astype(object)
w_nNR_21opt.head(n=5)

(669, 7)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres,MIN(wHazardChange)
0,1200741010700100571330001,3,102,-1.5,84.5,1998.1,-1.5
1,1200741010700100714590001,1,100,-0.5,76.8,1712.0,-0.5
2,1200741010700100717720001,21,202,-0.2,79.8,1712.0,-0.2
3,1200741010700100745730001,21,202,-0.2,61.1,1764.1,-0.2
4,1200741010700100814910001,1,100,-0.3,88.9,1799.2,-0.3


In [64]:
#Sum up the number of acres that can be treated with a 21" DBH cap.      
treat21=w_nNR_21opt.sum()["acres"]
print(treat21)

1354067.66252


Number of acres that can be treated by one or more packages with a 21"DBH cap: **1,354,067**. 

## Now do the same thing with 30 cap, how do they compare?

In [65]:
#filter optimiztion table by cap21 
w_nNR_30=w_nNR_opt[w_nNR_opt['rx'].isin(cap30)]

print(w_nNR_30.shape)
w_nNR_30.head(n=5)


(2772, 6)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres
2537,1200741010700100571330001,9,104,-0.3,83.6,1998.1
2538,1200741010700100600620001,9,104,-0.2,83.3,1820.5
2539,1200741010700100714590001,9,104,-0.2,86.1,1712.0
2540,1200741010700100745730001,9,104,-0.2,61.1,1764.1
2541,1200741010700100814910001,9,104,-0.5,85.1,1799.2


In [66]:
#Optimize by stand, max reduction in PTorch, create new table w_nNR_21opt

from pandasql import *

pysqldf = lambda q: sqldf(q, globals())

q3  = """
SELECT *, MIN(wHazardChange) FROM w_nNR_30 WHERE wHazardChange < 0 GROUP BY biosum_cond_id;
"""
w_nNR_30opt = pysqldf(q3)

print(w_nNR_30opt.shape)
w_nNR_30opt.biosum_cond_id = w_nNR_30opt.biosum_cond_id.astype(object)
w_nNR_30opt.head(n=5)

(692, 7)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres,MIN(wHazardChange)
0,1200741010700100571330001,11,106,-0.6,84.5,1998.1,-0.6
1,1200741010700100600620001,9,104,-0.2,83.3,1820.5,-0.2
2,1200741010700100714590001,23,204,-0.5,70.7,1712.0,-0.5
3,1200741010700100717720001,25,206,-0.2,79.8,1712.0,-0.2
4,1200741010700100745730001,9,104,-0.2,61.1,1764.1,-0.2


In [67]:
#Sum up the number of acres that can be treated with a 21" DBH cap.      
treat30=w_nNR_30opt.sum()["acres"]
print(treat30)
print (treat30-treat21)

1405320.19109
51252.5285704


Number of acres that can be treated by one or more packages with a 30"DBH cap: **1,405,320**. 

Compare this to 491,138 when the cap is 21"DBH, a difference of **51,252** acres. 

## How about when you restrict results to only acres with a NET REV > 0? 

In [68]:
#read in the cycle1_optimization created by SQL above from tables in core analysis
w_NR_opt = pd.read_excel(open('data/weighted_NR_opt.xlsx', 'rb'), sheetname='export', lowmemory=False)

w_NR_opt.biosum_cond_id = w_NR_opt.biosum_cond_id.astype(str)
w_NR_opt.rx = w_NR_opt.rx.astype(str)
w_NR_opt.rxpackage = w_NR_opt.rxpackage.astype(str)

In [69]:
#filter optimiztion table by cap21 
w_NR_21=w_NR_opt[w_NR_opt['rx'].isin(cap21)]

print(w_NR_21.shape)
w_NR_21.head(n=5)


(1540, 6)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres
15,1200741010700100745730001,21,202,-0.2,61.1,1764.1
18,1200741010700100814910001,1,100,-0.3,88.9,1799.2
28,1200741010702300706560001,3,102,-0.7,81.1,2131.6
29,1200741010702300706560001,4,103,-0.2,84.2,2131.6
35,1200741010702300706560001,21,202,-0.1,87.8,2131.6


In [70]:
#Optimize by stand, max reduction in PTorch, create new table w_nNR_21opt

from pandasql import *

pysqldf = lambda q: sqldf(q, globals())

q5  = """
SELECT *, MIN(wHazardChange) FROM w_NR_21 WHERE wHazardChange < 0 GROUP BY biosum_cond_id;
"""
w_NR_21opt = pysqldf(q5)

print(w_NR_21opt.shape)
w_NR_21opt.biosum_cond_id = w_NR_21opt.biosum_cond_id.astype(object)
w_NR_21opt.head(n=5)

(440, 7)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres,MIN(wHazardChange)
0,1200741010700100745730001,21,202,-0.2,61.1,1764.1,-0.2
1,1200741010700100814910001,1,100,-0.3,88.9,1799.2,-0.3
2,1200741010702300706560001,3,102,-0.7,81.1,2131.6,-0.7
3,1200741010702300724110001,1,100,-1.4,56.1,1997.9,-1.4
4,1200741010702300831920001,3,102,-0.6,86.5,2192.1,-0.6


In [71]:
#Sum up the number of acres that can be treated with a 21" DBH cap.      
treat21nr=w_NR_21opt.sum()["acres"]
print(treat21nr)

905175.68568


When net rev must be positive, the number of acres that can be treated with one or more rx with a cap of 21"DBH: **905,175**

## Now do the same thing with 30 cap, how do they compare?

In [72]:
#filter optimiztion table by cap21 
w_NR_30=w_NR_opt[w_NR_opt['rx'].isin(cap30)]

print(w_NR_30.shape)
w_NR_30.head(n=5)


(1918, 6)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres
0,1200741010700100571330001,9,104,-0.3,83.6,1998.1
1,1200741010700100571330001,10,105,-0.3,83.9,1998.1
2,1200741010700100571330001,11,106,-0.6,84.5,1998.1
3,1200741010700100571330001,12,107,-0.3,84.7,1998.1
6,1200741010700100571330001,23,204,-0.1,78.9,1998.1


In [73]:
#Optimize by stand, max reduction in PTorch, create new table w_NR_21opt

from pandasql import *

pysqldf = lambda q: sqldf(q, globals())

q7  = """
SELECT *, MIN (wHazardChange) FROM w_NR_30 WHERE wHazardChange < 0 GROUP BY biosum_cond_id;
"""
w_NR_30opt = pysqldf(q7)

print(w_NR_30opt.shape)
w_NR_30opt.biosum_cond_id = w_NR_30opt.biosum_cond_id.astype(object)
w_NR_30opt.head(n=5)

(554, 7)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres,MIN (wHazardChange)
0,1200741010700100571330001,11,106,-0.6,84.5,1998.1,-0.6
1,1200741010700100600620001,9,104,-0.2,83.3,1820.5,-0.2
2,1200741010700100745730001,9,104,-0.2,61.1,1764.1,-0.2
3,1200741010700100814910001,9,104,-0.5,85.1,1799.2,-0.5
4,1200741010700100884670001,24,205,-0.0,20.5,1820.5,-0.0


In [75]:
#Sum up the number of acres that can be treated with a 21" DBH cap.      
treat30nr=w_NR_30opt.sum()["acres"]
print(treat30nr)
print(treat30nr-treat21nr)

1147909.99927
242734.313586


When net rev must be positive, the number of acres that can be treated with one or more rx with a cap of 30"DBH: **1,147,909**

Compare to 905,175 with a 21" cap, a **242,734** acre difference. 


In [78]:
#create table of results
pd.set_option('display.float_format', lambda x: '%.0f' % x)
headings= ("NR", "21", "30")
n=("N", treat21, treat30)
y=("Y", treat21nr, treat30nr)

capData = [{'NR' : 'N', '21' : treat21, '30' : treat30},
          {'NR' : 'Y', '21' : treat21nr, '30' : treat30nr},
          {'NR' : 'PercentTotalNR', '21' : (treat21nr/total_acres)*100, '30' : (treat30nr/total_acres)*100},
          {'NR' : 'PercentTotalnNR', '21' : (treat21/total_acres)*100, '30' : (treat30/total_acres)*100}]
caps = pd.DataFrame(capData)
print(caps)

       21      30               NR
0 1354068 1405320                N
1  905176 1147910                Y
2      30      38   PercentTotalNR
3      45      47  PercentTotalnNR


# Is there any treatment benefit of cutting 0-5 inch trees? On how many acres? By how many composite score points? (9v34, 33v10)

  Old pairings - (1v2, 3v4, 9v10, 11v12, 13v14, 15v16, 19v20, 21v22, 23v24, 25v26, 27v28, 29v30, 31v32)

In [79]:
# list of rx values that coorespond 0 and 5 inch DBH minimums
min5 = ('9', '33')

min0 = ('34', '10')

In [80]:
#make a copy of the w_nNR_opt table created in the last inquiry
w_nNR_DBH = w_nNR_opt.copy(deep=True)
w_nNR_DBH.head(n=5)

,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres
0,1200741010700100571330001,1,100,-1,70,1998
1,1200741010700100714590001,1,100,-0,77,1712
2,1200741010700100814910001,1,100,-0,89,1799
3,1200741010700100880590002,1,100,-0,47,652
4,1200741010702300724110001,1,100,-1,56,1998


In [81]:
#Create minDBH column and assign 0 or 5 based on treatment
w_nNR_DBH.minDBH = w_nNR_DBH.ix[w_nNR_DBH.rxpackage.isin(min0), 'minDBH'] = '0'
w_nNR_DBH.minDBH = w_nNR_DBH.ix[w_nNR_DBH.rxpackage.isin(min5), 'minDBH'] = '5'

w_nNR_DBH.head(n=5)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres,minDBH
0,1200741010700100571330001,1,100,-1,70,1998,NaN
1,1200741010700100714590001,1,100,-0,77,1712,NaN
2,1200741010700100814910001,1,100,-0,89,1799,NaN
3,1200741010700100880590002,1,100,-0,47,652,NaN
4,1200741010702300724110001,1,100,-1,56,1998,NaN


In [82]:
#group by owner and hazard score, display acres
groupby_m = w_nNR_DBH[['rx','minDBH','acres']].groupby(['rx','minDBH'])

In [83]:
#calculate the sum of acre for minDBH groups, rxpackages and provide count of stands.
w_nNR_DBH = groupby_m.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .join(pd.DataFrame(groupby_m.size(), 
                                      columns=['counts']))\
                .reset_index()
            

w_nNR_DBH.head(n=5)

,rx,minDBH,acres sum,counts
0,104,5,1022114,509
1,105,0,538906,265
2,304,5,351585,171
3,305,0,951239,477


In [84]:
#find the difference in acres between min0 and min5 treatment potential. 
#create 'pair' column
#asssign a specific value to each treatment pair.  ie. treatments that are identical except for min DBH
w_nNR_DBH.pair = w_nNR_DBH.ix[w_nNR_DBH.rx.isin(['104', '305']), 'pair'] = 'A'
w_nNR_DBH.pair = w_nNR_DBH.ix[w_nNR_DBH.rx.isin(['105', '304']), 'pair'] = 'B'

w_nNR_DBH.head(n=5)

,rx,minDBH,acres sum,counts,pair
0,104,5,1022114,509,A
1,105,0,538906,265,B
2,304,5,351585,171,B
3,305,0,951239,477,A


In [85]:
#export for graphing in R
w_nNR_DBH.to_csv("outputs/w_nNR_DBH.csv")

In [86]:
#sort by twin and rxpackage
w_nNR_DBH.sort_values(['pair', 'minDBH'], ascending=True, inplace=True)
w_nNR_DBH.head(n=5)

,rx,minDBH,acres sum,counts,pair
3,305,0,951239,477,A
0,104,5,1022114,509,A
1,105,0,538906,265,B
2,304,5,351585,171,B


In [87]:
# find difference in acres between min0 and min5 in each pair
#group by minDBH and pair, display acres

#subtract min5 from min0 acre sums
w_nNR_DBH['diff'] = w_nNR_DBH['acres sum'] - w_nNR_DBH['acres sum'].shift(-1)

#remove the min5 rows from dataframe
w_nNR_DBHdiff = w_nNR_DBH[w_nNR_DBH['minDBH'] == '0']

#print first five rows
w_nNR_DBHdiff.head(n=5)

,rx,minDBH,acres sum,counts,pair,diff
3,305,0,951239,477,A,-70875
1,105,0,538906,265,B,187321


In [88]:
#get statistics for difference in acreage between pairs. 
w_nNR_DBHdiff['diff'].describe()

count        2
mean     58223
std     182572
min     -70875
25%      -6326
50%      58223
75%     122772
max     187321
Name: diff, dtype: float64

In [89]:
Image(url= "graphs/minDBH.png", width=500, height=400)

In [90]:
Image(url= "graphs/minDBH2.png", width=500, height=400)

# Pick a residual BA and dbh cap (repeat for a couple other combos). For the set of plots where treatments are implementable in both cases, what is the average difference in effectiveness (score diff or difference in score relative to grow only) for thin across vs thin below? What is the difference in average net revenue? (1-4 = 19-22, 9-12 = 23-26)

Use these queries in the weighted_nNR core analysis scenario DB

``` mysql
SELECT cycle1_effective.biosum_cond_id, cycle1_effective.rxpackage, cycle1_effective.rx, cycle1_effective.rxcycle, cycle1_effective.pre_variable1_name, cycle1_effective.post_variable1_name, cycle1_effective.variable1_change, cycle1_effective.variable1_effective_yn, cond.acres INTO question8
FROM cond INNER JOIN cycle1_effective ON cond.biosum_cond_id = cycle1_effective.biosum_cond_id;

UPDATE product_yields_net_rev_costs_summary_by_rxpackage INNER JOIN question8 ON (product_yields_net_rev_costs_summary_by_rxpackage.rxpackage = question8.rxpackage) AND (product_yields_net_rev_costs_summary_by_rxpackage.biosum_cond_id = question8.biosum_cond_id) SET question8.max_nr_dpa = [product_yields_net_rev_costs_summary_by_rxpackage].[max_nr_dpa];


```

In [91]:
#read in the enter table created by SQL above from queries run in core analysis database
w_nNR_enter = pd.read_excel(open('data/enter_noNR.xlsx', 'rb'), sheetname='question8', lowmemory=False)
#change a couple columns to string
w_nNR_enter.biosum_cond_id = w_nNR_enter.biosum_cond_id.astype(str)
w_nNR_enter.rx = w_nNR_enter.rx.astype(str)
w_nNR_enter.rxpackage = w_nNR_enter.rxpackage.astype(str)
#print first five records of new table
w_nNR_enter.head(n=5)

,biosum_cond_id,rxpackage,rx,rxcycle,pre_variable1_name,post_variable1_name,variable1_change,variable1_effective_yn,acres,max_nr_dpa
0,1200641050705900795510001,15,110,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,1,N,7336,-640
1,1200741010700100522500001,1,100,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,nan,NaN,1812,-153
2,1200741010700100571330001,1,100,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2778
3,1200741010700100571330001,2,101,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2367
4,1200741010700100571330001,3,102,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-730


In [92]:
# find set of plots where treatements are implementable in both cases

#create a column 'twin'
#assign a 'twin' value to each pair of treatments that are identical except for thin type
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['1', '19']), 'twin'] = 'A'
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['2', '20']), 'twin'] = 'B'
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['3', '21']), 'twin'] = 'C'
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['4', '22']), 'twin'] = 'D'
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['9', '23']), 'twin'] = 'E'
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['10', '24']), 'twin'] = 'F'
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['11', '25']), 'twin'] = 'G'
w_nNR_enter.twin = w_nNR_enter.ix[w_nNR_enter.rxpackage.isin(['12', '26']), 'twin'] = 'H'

w_nNR_enter.head(n=5)


,biosum_cond_id,rxpackage,rx,rxcycle,pre_variable1_name,post_variable1_name,variable1_change,variable1_effective_yn,acres,max_nr_dpa,twin
0,1200641050705900795510001,15,110,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,1,N,7336,-640,NaN
1,1200741010700100522500001,1,100,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,nan,NaN,1812,-153,A
2,1200741010700100571330001,1,100,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2778,A
3,1200741010700100571330001,2,101,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2367,B
4,1200741010700100571330001,3,102,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-730,C


In [93]:
#groupby biosumID and twin group
groupby_T = w_nNR_enter[['biosum_cond_id', 'twin','acres']].groupby(['biosum_cond_id', 'twin'])
#get a count of the number of records in each twin group.
w_nNR_enterA = groupby_T.agg('min')\
                .rename(columns = lambda x: x)\
                .join(pd.DataFrame(groupby_T.size(), 
                                      columns=['counts']))
#filter for biosumID/package groups that have a count of 2, meaning the stand could be entered by each package in the twin group. 
w_nNR_enterA=pd.DataFrame(w_nNR_enterA.loc[w_nNR_enterA['counts'] == 2])            
w_nNR_enterA.reset_index(inplace=True)  
w_nNR_enterA.head(n=5)

,biosum_cond_id,twin,acres,counts
0,1200741010700100571330001,A,1998,2
1,1200741010700100571330001,B,1998,2
2,1200741010700100571330001,C,1998,2
3,1200741010700100571330001,D,1998,2
4,1200741010700100571330001,E,1998,2


In [94]:
#create a new table that only contains the records for stands/packages that could be entered by both packages in the twin
#merge with original table to get the variable values and max_nr values. 
enter = pd.merge(w_nNR_enter, w_nNR_enterA, how='inner', on=['biosum_cond_id', 'twin'])
enter.to_csv('outputs/enter.csv')
enter.head(n=5)
#check out max_nr graph in R_graphing, link below

,biosum_cond_id,rxpackage,rx,rxcycle,pre_variable1_name,post_variable1_name,variable1_change,variable1_effective_yn,acres_x,max_nr_dpa,twin,acres_y,counts
0,1200741010700100571330001,1,100,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2778,A,1998,2
1,1200741010700100571330001,19,200,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2393,A,1998,2
2,1200741010700100571330001,2,101,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2367,B,1998,2
3,1200741010700100571330001,20,201,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,-2299,B,1998,2
4,1200741010700100571330001,3,102,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-730,C,1998,2


In [95]:
#expand max_nr_dpa by acres, name expansion
enter['expansion'] = enter['max_nr_dpa'] * enter['acres_x']  # assigned to a column
enter.to_csv('outputs/enter.csv')
enter.head(n=5)

,biosum_cond_id,rxpackage,rx,rxcycle,pre_variable1_name,post_variable1_name,variable1_change,variable1_effective_yn,acres_x,max_nr_dpa,twin,acres_y,counts,expansion
0,1200741010700100571330001,1,100,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2778,A,1998,2,-5550248
1,1200741010700100571330001,19,200,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2393,A,1998,2,-4782376
2,1200741010700100571330001,2,101,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-2367,B,1998,2,-4729747
3,1200741010700100571330001,20,201,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,-2299,B,1998,2,-4593687
4,1200741010700100571330001,3,102,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-1,Y,1998,-730,C,1998,2,-1459592


In [97]:
#table that calculates the total entered acres for each pair of treatments 
#acres that could be entered by both packages. 
enterTotalAcres=enter.groupby(['twin','rx']).agg('sum').rename(columns = lambda x: x + '_sum').reset_index()

In [98]:
#create a table that contains only effective = Y sums for each twin/rxpackage

#filter by effective = Y
enterY=pd.DataFrame(enter.loc[enter['variable1_effective_yn'] == 'Y'])  

enterY.rx =enterY.rx.astype(int)
#sort by twin and rxpackage
enterY.sort_values(['twin', 'rx'], ascending=True, inplace=True)

#enterY.to_csv("outputs/enterY.csv"


In [99]:
enterYsub=enterY[['rx', 'twin', 'max_nr_dpa']]
enterYsub.to_csv("outputs/enterYsub.csv")
enterYsub.head(n=3)

,rx,twin,max_nr_dpa
0,100,A,-2778
88,100,A,-2935
148,100,A,1026


In [100]:
#group by twin and variable effective yn , display acres
groupby_e = enterY[['twin', 'rx', 'variable1_effective_yn','variable1_change', 'acres_x', 'max_nr_dpa', 'expansion']].groupby(['twin', 'rx','variable1_effective_yn'])


In [101]:
#calculate the sum of acre for minDBH groups, rxpackages and provide count of stands.
enterAcres = groupby_e.agg('sum')\
                .rename(columns = lambda x: x + '_sum')\
                .reset_index()
enterAcres.head(n=5)

,twin,rx,variable1_effective_yn,variable1_change_sum,acres_x_sum,max_nr_dpa_sum,expansion_sum
0,A,100,Y,-335,949877,355571,679221187
1,A,200,Y,-394,931084,65515,102884577
2,B,101,Y,-138,574785,179762,347822851
3,B,201,Y,-188,685758,159728,272496703
4,C,102,Y,-253,810523,296307,577139667


In [102]:
#divided net revenue sum by the total acres for that rx
enterAcres['avgMaxNR'] = enterAcres['expansion_sum'] / enterAcres['acres_x_sum']  # assigned to a column
enterAcres.head(n=5)

,twin,rx,variable1_effective_yn,variable1_change_sum,acres_x_sum,max_nr_dpa_sum,expansion_sum,avgMaxNR
0,A,100,Y,-335,949877,355571,679221187,715
1,A,200,Y,-394,931084,65515,102884577,110
2,B,101,Y,-138,574785,179762,347822851,605
3,B,201,Y,-188,685758,159728,272496703,397
4,C,102,Y,-253,810523,296307,577139667,712


In [103]:
#export to csv
enterAcres.rx =enterAcres.rx.astype(int)
enterTotalAcres.rx =enterTotalAcres.rx.astype(int)
enterAcresY = pd.merge(enterAcres, enterTotalAcres, how='inner', on=['twin', 'rx'])
enterAcresY.to_csv("outputs/enterAcresY.csv")


In [105]:
# find difference in acres between twin thin types

enterAcres.rx =enterAcres.rx.astype(int)
#sort by twin and rxpackage
enterAcres.sort_values(['twin', 'rx'], ascending=True, inplace=True)

#subtract min5 from thinbba from thindbh acre sums
enterAcres['diff'] = enterAcres['acres_x_sum'] - enterAcres['acres_x_sum'].shift(-1)
enterAcres['diff_nr'] = enterAcres['avgMaxNR'] - enterAcres['avgMaxNR'].shift(-1)

#remove the thinbba rows from dataframe
enterYdiff = enterAcres[enterAcres['rx'] < 200]

#print first five rows 
#contains the difference in acres effectively treated and net revenue.  
#diff = ThinDBH acres effectively treated - ThinBBA acres effectively treated
#idff_nr = ThinDBH max nr of acres effectively treated - ThinBBA max nr of acres effectively treated
enterYdiff.head(n=5)

,twin,rx,variable1_effective_yn,variable1_change_sum,acres_x_sum,max_nr_dpa_sum,expansion_sum,avgMaxNR,diff,diff_nr
0,A,100,Y,-335,949877,355571,679221187,715,18793,605
2,B,101,Y,-138,574785,179762,347822851,605,-110973,208
4,C,102,Y,-253,810523,296307,577139667,712,7972,773
6,D,103,Y,-89,380505,137243,251311369,660,-116115,383
8,E,104,Y,-335,1022114,1181747,2334315682,2284,152651,1005


In [106]:
enterYdiff['diff_nr'].describe()

count      8
mean     897
std      529
min      208
25%      549
50%      889
75%     1107
max     1854
Name: diff_nr, dtype: float64

In [107]:
enterYdiff['diff'].describe()

count         8
mean     -34581
std      100718
min     -144138
25%     -112258
50%      -46129
75%       16242
max      152651
Name: diff, dtype: float64

In [108]:
Image(url= "graphs/DBH_treat.png", width=700, height=400)

In [109]:
Image(url= "graphs/DBH_dpa.png", width=700, height=400)

In [110]:
Image(url= "graphs/DHB_NR.png", width=500, height=250)

On acres where both treatments are effective, what is the distribution of hazard score change? 

In [111]:
Image(url= "graphs/DBH_hDiff.png", width=500, height=250)

# Does targeting grand fir make any difference to effectiveness, on average? Need to limit to only those stands where there were enough grand fir present for the grand fir cut rate to be different under the cut grand fir first policy. (9-10 v 13-14, 23-24 v 27v28)

Select the stand ID's for which the Grand Fir BA is greater than the target grand fir BA (and therefore firs would be cut first).  This query was used in the PREPOST_COMPUTE database to create that list of stands. 

```mysql
SELECT PRE_FVS_COMPUTE.biosum_cond_id
FROM PRE_FVS_COMPUTE
WHERE (((PRE_FVS_COMPUTE.rxpackage)="001") AND ((PRE_FVS_COMPUTE.rxcycle)="1") AND ((PRE_FVS_COMPUTE.GFBA)>[PRE_FVS_COMPUTE].[TBA]));
```

In [112]:
#read in the GF table created by the above query
GF = pd.read_excel(open('data/GF.xlsx', 'rb'), sheetname='GF', lowmemory=False)
#change ID column to string
GF.biosum_cond_id = GF.biosum_cond_id.astype(str)
#print first five records of new table
GF.head(n=5)

,biosum_cond_id
0,1200741010700100571330001
1,1200741010700100600620001
2,1200741010700100745730001
3,1200741010700100814910001
4,1200741010700100935520001


In [113]:
#read in the enter table enter_noNR table.
#table contains all entered stands, effective treatment or not
gfEnter = pd.read_excel(open('data/enter_noNR.xlsx', 'rb'), sheetname='question8', lowmemory=False)
#change a couple columns to string
gfEnter.biosum_cond_id = gfEnter.biosum_cond_id.astype(str)
gfEnter.rx = gfEnter.rx.astype(str)
gfEnter.rxpackage = gfEnter.rxpackage.astype(str)


In [114]:
#create a column 'duo'
#assign a 'dup' value to each pair of treatments that are identical except for target/not target GF
gfEnter.duo = gfEnter.ix[gfEnter.rxpackage.isin(['9', '13']), 'duo'] = 'A'
gfEnter.duo = gfEnter.ix[gfEnter.rxpackage.isin(['10', '14']), 'duo'] = 'B'
gfEnter.duo = gfEnter.ix[gfEnter.rxpackage.isin(['23', '27']), 'duo'] = 'C'
gfEnter.duo = gfEnter.ix[gfEnter.rxpackage.isin(['24', '28']), 'duo'] = 'D'

#drop rows without a duo value, we don't care about them right now. 
gfEnter = gfEnter[pd.notnull(gfEnter['duo'])]

gfEnter.to_csv('outputs/gfEnter.csv')


In [115]:
#create a new table that only contains the records for stands with GF that would have been targeted for first cut
#merge with original table to get the variable values and max_nr values. 
gf_p = pd.merge(gfEnter, GF, how='inner', on=['biosum_cond_id'])
gf_p.head(n=5)

,biosum_cond_id,rxpackage,rx,rxcycle,pre_variable1_name,post_variable1_name,variable1_change,variable1_effective_yn,acres,max_nr_dpa,duo
0,1200741010700100571330001,9,104,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2252,A
1,1200741010700100571330001,10,105,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2352,B
2,1200741010700100571330001,13,108,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2070,A
3,1200741010700100571330001,14,109,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2342,B
4,1200741010700100571330001,23,204,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,1450,C


In [116]:
#expand max_nr_dpa by acres, name expansion
gf_p['expansion'] = gf_p['max_nr_dpa'] * gf_p['acres']  # assigned to a column
gf_p.head(n=5)

,biosum_cond_id,rxpackage,rx,rxcycle,pre_variable1_name,post_variable1_name,variable1_change,variable1_effective_yn,acres,max_nr_dpa,duo,expansion
0,1200741010700100571330001,9,104,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2252,A,4500170
1,1200741010700100571330001,10,105,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2352,B,4699979
2,1200741010700100571330001,13,108,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2070,A,4135588
3,1200741010700100571330001,14,109,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,2342,B,4679724
4,1200741010700100571330001,23,204,1,PRE_FVS_POTFIRE.wHazard,POST_FVS_POTFIRE.wHazard,-0,Y,1998,1450,C,2897898


In [117]:
#table that calculates the total entered acres for each pair of treatments 
#acres that could be entered by both packages. 
enterTotalAcresGF=gf_p.groupby(['duo','rx']).agg('sum').rename(columns = lambda x: x + '_sum').reset_index()

In [118]:
#create a table that contains only effective = Y sums for each twin/rxpackage

#filter by effective = Y
gf_y=pd.DataFrame(gf_p.loc[gf_p['variable1_effective_yn'] == 'Y'])  

gf_y.rx =gf_y.rx.astype(int)
#sort by twin and rxpackage
gf_y.sort_values(['duo', 'rx'], ascending=True, inplace=True)

#enterY.to_csv("outputs/enterY.csv")


In [119]:
#group by twin and variable effective yn , display acres
groupby_gf = gf_y[['duo', 'rx', 'variable1_effective_yn','variable1_change', 'acres', 'max_nr_dpa', 'expansion']].groupby(['duo', 'rx','variable1_effective_yn'])


In [120]:
#calculate the sum of acre for minDBH groups, rxpackages and provide count of stands.
enterGF = groupby_gf.agg('sum')\
                .rename(columns = lambda x: x + '_sum')\
                .reset_index()
enterGF.head(n=5)

,duo,rx,variable1_effective_yn,variable1_change_sum,acres_sum,max_nr_dpa_sum,expansion_sum
0,A,104,Y,-232,658951,922224,1789980210
1,A,108,Y,-231,576631,569753,1106858141
2,B,105,Y,-80,363580,506924,993295352
3,B,109,Y,-79,377512,532904,989480662
4,C,204,Y,-248,572461,501671,907936167


In [122]:
#divided net revenue sum by the total acres for that rx
enterGF['avgMaxNR'] = enterGF['expansion_sum'] / enterGF['acres_sum']  # assigned to a column
enterGF.head(n=3)

,duo,rx,variable1_effective_yn,variable1_change_sum,acres_sum,max_nr_dpa_sum,expansion_sum,avgMaxNR
0,A,104,Y,-232,658951,922224,1789980210,2716
1,A,108,Y,-231,576631,569753,1106858141,1920
2,B,105,Y,-80,363580,506924,993295352,2732


In [124]:
#export to csv for graphing in R
enterGF.rx =enterGF.rx.astype(int)
enterTotalAcresGF.rx =enterTotalAcresGF.rx.astype(int)
enterAcresGF = pd.merge(enterGF, enterTotalAcresGF, how='inner', on=['duo', 'rx'])
enterAcresGF.to_csv("outputs/enterAcresGF.csv")

enterAcresGF.head(n=1)

,duo,rx,variable1_effective_yn,variable1_change_sum_x,acres_sum_x,max_nr_dpa_sum_x,expansion_sum_x,avgMaxNR,rxcycle_sum,variable1_change_sum_y,acres_sum_y,max_nr_dpa_sum_y,expansion_sum_y
0,A,104,Y,-232,658951,922224,1789980210,2716,529,-121,1101149,1344125,2652177323


In [125]:
# find difference in acres between twin thin types

enterAcresGF.rx =enterAcresGF.rx.astype(int)
#sort by twin and rxpackage
enterAcresGF.sort_values(['duo', 'rx'], ascending=True, inplace=True)

#subtract min5 from thinbba from thindbh acre sums
enterAcresGF['diff'] = enterAcresGF['acres_sum_x'] - enterAcresGF['acres_sum_x'].shift(-1)
enterAcresGF['diff_nr'] = enterAcresGF['avgMaxNR'] - enterAcresGF['avgMaxNR'].shift(-1)

#remove 
gf_Ydiff =  enterAcresGF.query("rx == 104 or rx == 105 or rx == 204 or rx == 205")

#print first five rows 
gf_Ydiff.head(n=20)

,duo,rx,variable1_effective_yn,variable1_change_sum_x,acres_sum_x,max_nr_dpa_sum_x,expansion_sum_x,avgMaxNR,rxcycle_sum,variable1_change_sum_y,acres_sum_y,max_nr_dpa_sum_y,expansion_sum_y,diff,diff_nr
0,A,104,Y,-232,658951,922224,1789980210,2716,529,-121,1101149,1344125,2652177323,82320,797
2,B,105,Y,-80,363580,506924,993295352,2732,529,166,1101149,1181431,2295240069,-13931,111
4,C,204,Y,-248,572461,501671,907936167,1586,538,-65,1123445,939356,1775234499,-22307,-153
6,D,205,Y,-101,436334,259422,508923571,1166,520,110,1085849,500582,984177962,43245,-404


In [126]:
gf_Ydiff['diff_nr'].describe()

count      4
mean      88
std      518
min     -404
25%     -216
50%      -21
75%      282
max      797
Name: diff_nr, dtype: float64

In [127]:
gf_Ydiff['diff'].describe()

count        4
mean     22332
std      49476
min     -22307
25%     -16025
50%      14657
75%      53014
max      82320
Name: diff, dtype: float64

In [131]:
Image(url= "graphs/GF_acres.png", width=400, height=250)

In [130]:
Image(url= "graphs/GF_dpa.png", width=500, height=250)

In [128]:
Image(url= "graphs/GF_NR.png", width=500, height=250)

On the acres where both treatments are effecitve, what is the distrubiton of hazard score change? 

In [132]:
Image(url= "graphs/GF_hdiff.png", width=500, height=250)

# What difference does style of surface fuel treatment make? 

In [133]:
# list of rx values that coorespond 0 and 5 inch DBH minimums
l_s = ('10', '33')

burn = ('34', '9')

In [134]:
#read in the cycle1_optimization created by SQL above from tables in core analysis
w_nNR_fuel = pd.read_excel(open('data/weighted_noNR_opt2.xlsx', 'rb'), sheetname='export', lowmemory=False)

w_nNR_fuel.biosum_cond_id = w_nNR_opt.biosum_cond_id.astype(str)
w_nNR_fuel.rx = w_nNR_opt.rx.astype(str)
w_nNR_fuel.rxpackage = w_nNR_opt.rxpackage.astype(str)

In [135]:
#Create minDBH column and assign 0 or 5 based on treatment
w_nNR_fuel.fuel = w_nNR_fuel.ix[w_nNR_fuel.rxpackage.isin(l_s), 'fuel'] = 'ls'
w_nNR_fuel.fuel = w_nNR_fuel.ix[w_nNR_fuel.rxpackage.isin(burn), 'fuel'] = 'burn'

w_nNR_fuel.head(n=5)


,biosum_cond_id,rxpackage,rx,wHazardChange,MVPPost,acres,fuel
0,1200741010700100571330001,1,100,-1,70,1998,NaN
1,1200741010700100714590001,1,100,-0,77,1712,NaN
2,1200741010700100814910001,1,100,-0,89,1799,NaN
3,1200741010700100880590002,1,100,-0,47,652,NaN
4,1200741010702300724110001,1,100,-1,56,1998,NaN


In [137]:
#group by owner and hazard score, display acres
groupby_f = w_nNR_fuel[['rx','fuel','acres']].groupby(['rx','fuel'])

In [138]:
#calculate the sum of acre for minDBH groups, rxpackages and provide count of stands.
w_nNR_sf = groupby_f.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .join(pd.DataFrame(groupby_f.size(), 
                                      columns=['counts']))\
                .reset_index()
            

w_nNR_sf.head(n=5)

,rx,fuel,acres sum,counts
0,104,burn,1022114,509
1,105,ls,538906,265
2,304,ls,351585,171
3,305,burn,951239,477


In [139]:
#find the difference in acres between min0 and min5 treatment potential. 
#create 'pair' column
#asssign a specific value to each treatment pair.  ie. treatments that are identical except for min DBH
w_nNR_sf.pair = w_nNR_sf.ix[w_nNR_sf.rx.isin(['104', '304']), 'pair'] = 'A'
w_nNR_sf.pair = w_nNR_sf.ix[w_nNR_sf.rx.isin(['105', '305']), 'pair'] = 'B'

w_nNR_sf.head(n=5)

,rx,fuel,acres sum,counts,pair
0,104,burn,1022114,509,A
1,105,ls,538906,265,B
2,304,ls,351585,171,A
3,305,burn,951239,477,B


In [140]:
#export to csv for graphing in R
w_nNR_sf.to_csv("outputs/w_nNR_sf.csv")

In [141]:
#sort by twin and rxpackage
w_nNR_sf.sort_values(['pair', 'fuel'], ascending=True, inplace=True)
w_nNR_sf.head(n=5)

,rx,fuel,acres sum,counts,pair
0,104,burn,1022114,509,A
2,304,ls,351585,171,A
3,305,burn,951239,477,B
1,105,ls,538906,265,B


In [142]:
# find difference in acres between min0 and min5 in each pair
#group by minDBH and pair, display acres

#subtract min5 from min0 acre sums
w_nNR_sf['diff'] = w_nNR_sf['acres sum'] - w_nNR_sf['acres sum'].shift(-1)

#remove the min5 rows from dataframe
w_nNR_sfdiff = w_nNR_sf[w_nNR_sf['fuel'] == 'burn']

#print first five rows
w_nNR_sfdiff.head(n=5)

,rx,fuel,acres sum,counts,pair,diff
0,104,burn,1022114,509,A,670529
3,305,burn,951239,477,B,412333


In [143]:
#get statistics for difference in acreage between pairs. 
w_nNR_sfdiff['diff'].describe()

count        2
mean    541431
std     182572
min     412333
25%     476882
50%     541431
75%     605980
max     670529
Name: diff, dtype: float64

In [144]:
Image(url= "graphs/SF_treat.png", width=400, height=250)

In [145]:
Image(url= "graphs/SF2.png", width=400, height=250)

# How do log length treatments compare to WT ones in terms of cost? IN terms of revenues? Is the extra net revenue from selling more chips (and perhaps implementing a cheaper surface fuel treatment) offsetting the extra cost of doing whole tree harvest (if in fact WT is more expensive)? (1-4 vs. 29-32)

In [146]:
#read in the product cost/rev table from core analysis for rxpackage 
money = pd.read_excel(open('data/product.xlsx', 'rb'), sheetname='product', lowmemory=False)
#change a couple columns to string
money.biosum_cond_id = money.biosum_cond_id.astype(str)
money.rxpackage = money.rxpackage.astype(str)
#print first five records of table
money.head(n=5)

,biosum_cond_id,rxpackage,chip_yield_cf,merch_yield_cf,chip_yield_gt,merch_yield_gt,chip_val_dpa,merch_val_dpa,harvest_onsite_cpa,haul_chip_cpa,haul_merch_cpa,merch_chip_nr_dpa,merch_nr_dpa,max_nr_dpa
0,1200641050705900795510001,15,267,514,6,11,107,1118,1748,64,53,-640,-683,-640
1,1200741010700100522500001,1,506,871,10,18,184,1947,2048,77,159,-153,-260,-153
2,1200741010700100571330001,1,429,946,10,22,180,1924,4801,25,56,-2778,-2933,-2778
3,1200741010700100571330001,2,449,994,10,23,189,2016,4487,26,59,-2367,-2530,-2367
4,1200741010700100571330001,3,367,755,9,18,154,1549,2367,21,45,-730,-863,-730


In [147]:
#create a column 'twofer'
#assign a 'twofer' value to each pair of treatments that are identical except for harvest type
money.twofer = money.ix[money.rxpackage.isin(['1', '29']), 'twofer'] = 'A'
money.twofer = money.ix[money.rxpackage.isin(['2', '30']), 'twofer'] = 'B'
money.twofer = money.ix[money.rxpackage.isin(['3', '31']), 'twofer'] = 'C'
money.twofer = money.ix[money.rxpackage.isin(['4', '32']), 'twofer'] = 'D'

#drop rows without a twofer value, we don't care about them right now. 
moneyN = money[pd.notnull(money['twofer'])]

moneyN.head(n=5)

,biosum_cond_id,rxpackage,chip_yield_cf,merch_yield_cf,chip_yield_gt,merch_yield_gt,chip_val_dpa,merch_val_dpa,harvest_onsite_cpa,haul_chip_cpa,haul_merch_cpa,merch_chip_nr_dpa,merch_nr_dpa,max_nr_dpa,twofer
1,1200741010700100522500001,1,506,871,10,18,184,1947,2048,77,159,-153,-260,-153,A
2,1200741010700100571330001,1,429,946,10,22,180,1924,4801,25,56,-2778,-2933,-2778,A
3,1200741010700100571330001,2,449,994,10,23,189,2016,4487,26,59,-2367,-2530,-2367,B
4,1200741010700100571330001,3,367,755,9,18,154,1549,2367,21,45,-730,-863,-730,C
5,1200741010700100571330001,4,672,972,16,23,283,1979,4738,39,58,-2572,-2817,-2572,D


In [148]:
#get a count of the number of records in each group.
groupby_tf = moneyN.groupby(['biosum_cond_id', 'twofer']).size().reset_index(name='count')
#filter for biosumID/package groups that have a count of 2, meaning the stand could be entered by each package in the group. 
groupby_tf=pd.DataFrame(groupby_tf.loc[groupby_tf['count'] == 2])            
groupby_tf.reset_index(inplace=True) 
groupby_tf.head(n=2)

,index,biosum_cond_id,twofer,count
0,1,1200741010700100571330001,A,2
1,2,1200741010700100571330001,B,2


In [149]:
#create a new table that only contains the records for stands/packages that could be entered by both packages in the twin
#merge with original table to get the variable values and max_nr values. 
moneyN = pd.merge(moneyN, groupby_tf, how='inner', on=['biosum_cond_id', 'twofer'])

#check out max_nr graph in R_graphing, link below

In [150]:
#gotta change rxpackage for sorting
moneyN.rxpackage = moneyN.rxpackage.astype(int)
#sort by twofer and rxpackage
moneyN.sort_values(['biosum_cond_id', 'twofer', 'rxpackage'], ascending=True, inplace=True)

moneyN.head(n=5)

,biosum_cond_id,rxpackage,chip_yield_cf,merch_yield_cf,chip_yield_gt,merch_yield_gt,chip_val_dpa,merch_val_dpa,harvest_onsite_cpa,haul_chip_cpa,haul_merch_cpa,merch_chip_nr_dpa,merch_nr_dpa,max_nr_dpa,twofer,index,count
0,1200741010700100571330001,1,429,946,10,22,180,1924,4801,25,56,-2778,-2933,-2778,A,1,2
1,1200741010700100571330001,29,37,946,1,22,16,1924,4774,2,56,-2893,-2907,-2893,A,1,2
2,1200741010700100571330001,2,449,994,10,23,189,2016,4487,26,59,-2367,-2530,-2367,B,2,2
3,1200741010700100571330001,30,37,994,1,23,16,2016,4726,2,59,-2756,-2770,-2756,B,2,2
4,1200741010700100571330001,3,367,755,9,18,154,1549,2367,21,45,-730,-863,-730,C,3,2


In [151]:
# find difference in harvest cost between CTL and WT harvest systems

#subtract CTL from WT harvest costs
moneyN['diff_cost'] = moneyN['harvest_onsite_cpa'] - moneyN['harvest_onsite_cpa'].shift(-1)
moneyN['diff_cf'] = moneyN['chip_yield_cf'] - moneyN['chip_yield_cf'].shift(-1)
moneyN['diff_nr'] = moneyN['max_nr_dpa'] - moneyN['max_nr_dpa'].shift(-1)

moneyN.to_csv('outputs/moneyN.csv')

#remove the CTL rows from dataframe
moneyN = moneyN[moneyN['rxpackage'] < 5]


#print first five rows 
#contains the difference in acres effectively treated and net revenue.  
#diff_cost = WT cost - CTL cost 
#dfff_cf = WT chip_yield_cf - CTL chip_yield_cf
#dfff_nr = WT max net rev - CTL max net rev


In [152]:
moneyN.head(n=1)

,biosum_cond_id,rxpackage,chip_yield_cf,merch_yield_cf,chip_yield_gt,merch_yield_gt,chip_val_dpa,merch_val_dpa,harvest_onsite_cpa,haul_chip_cpa,haul_merch_cpa,merch_chip_nr_dpa,merch_nr_dpa,max_nr_dpa,twofer,index,count,diff_cost,diff_cf,diff_nr
0,1200741010700100571330001,1,429,946,10,22,180,1924,4801,25,56,-2778,-2933,-2778,A,1,2,27,392,116


In [153]:

#group by twin and variable effective yn , display acres
groupby_money = moneyN[['twofer', 'rxpackage', 'diff_cost','diff_cf', 'diff_nr']].groupby(['twofer', 'rxpackage'])
#aggregate records by rxpackage
money_sum = groupby_money.agg('mean')\
                .rename(columns = lambda x: x + '_mean')\
                .reset_index()

money_sum.head(n=5)

,twofer,rxpackage,diff_cost_mean,diff_cf_mean,diff_nr_mean
0,A,1,-918,913,1145
1,B,2,-1231,923,1446
2,C,3,-746,775,946
3,D,4,-863,778,1049


In [154]:
#get statistics for differences
money_sum['diff_cost_mean'].describe()

count       4
mean     -940
std       207
min     -1231
25%      -996
50%      -891
75%      -834
max      -746
Name: diff_cost_mean, dtype: float64

In [155]:
#get statistics for differences
moneyN['diff_cf'].describe()

count   4003
mean     854
std      517
min       14
25%      513
50%      705
75%     1080
max     6923
Name: diff_cf, dtype: float64

In [156]:
#get statistics for differences
moneyN['diff_nr'].describe()

count    4003
mean     1160
std      1001
min     -1313
25%       484
50%       930
75%      1585
max     13883
Name: diff_nr, dtype: float64

In [157]:
Image(url= "graphs/harv.png", width=400, height=250)

In [159]:
Image(url= "graphs/harv2.png", width=400, height=250)

# How much wood and energy produced? 

Using the selected 'best' treatment for each stand, how much wood is produced? Bioenergy?  On how many stands does harvesting energy wood increase the max_nr? What are the total treatment and haul costs? 

In [160]:
#read in the product cost/rev table from core analysis for rxpackage for the 'best' treatments (weighted average)
product = pd.read_excel(open('data/effective_products.xlsx', 'rb'), sheetname='effective_products', lowmemory=False)
#change a couple columns to string
product.biosum_cond_id = money.biosum_cond_id.astype(str)
product.rxpackage = money.rxpackage.astype(str)
#print first five records of table
product.head(n=5)

,biosum_cond_id,rxpackage,chip_yield_cf,merch_yield_cf,chip_yield_gt,merch_yield_gt,chip_val_dpa,merch_val_dpa,harvest_onsite_cpa,haul_chip_cpa,haul_merch_cpa,merch_chip_nr_dpa,merch_nr_dpa,max_nr_dpa
0,1200641050705900795510001,15,367,755,9,18,154,1549,2367,21,45,-730,-863,-730
1,1200741010700100522500001,1,426,3064,10,67,176,7220,3625,129,989,2653,2606,2653
2,1200741010700100571330001,1,603,841,13,18,236,1855,5323,78,74,-3383,-3542,-3383
3,1200741010700100571330001,2,603,841,13,18,236,1855,5323,78,74,-3383,-3542,-3383
4,1200741010700100571330001,3,987,464,22,9,390,946,3095,199,83,-2041,-2232,-2041


In [161]:
#calculate the sum of all rows

#turn off pesky scientific notation
pd.set_option('display.float_format', lambda x: '%.1f' % x)
#perform sum
product.sum(axis=0)

biosum_cond_id             inf
rxpackage                  inf
chip_yield_cf        1699017.6
merch_yield_cf       2941525.3
chip_yield_gt          35568.3
merch_yield_gt         62834.0
chip_val_dpa          640230.2
merch_val_dpa        6410712.1
harvest_onsite_cpa   5602252.3
haul_chip_cpa         247441.9
haul_merch_cpa        421612.9
merch_chip_nr_dpa     779635.2
merch_nr_dpa          386846.9
max_nr_dpa            781827.9
dtype: float64

In [162]:
#We can also bring in the stand cost/rev table to get the expanded (by acre results)
#read in the stand cost/rev table from core analysis for rxpackage for the 'best' treatments (weighted average)
stand = pd.read_excel(open('data/effective_stand.xlsx', 'rb'), sheetname='effective_stand', lowmemory=False)
#change a couple columns to string
stand.biosum_cond_id = money.biosum_cond_id.astype(str)
stand.rxpackage = money.rxpackage.astype(str)
#print first five records of table
stand.head(n=5)

,biosum_cond_id,rxpackage,owngrpcd,acres,merch_haul_cost_psite,merch_haul_cost_sum,merch_vol_cf_sum,merch_dollars_val_sum,chip_haul_cost_psite,chip_haul_cost_sum,chip_yield_gt_sum,chip_dollars_val_sum,net_rev_dollars_sum,harv_costs_sum,haul_costs_sum
0,1200641050705900795510001,15,10,1998.1,580,89337.8,1508468.9,3094140.6,593,41820.8,17067.0,307205.7,-1459592.3,4729780.0,131158.6
1,1200741010700100522500001,1,10,1820.5,580,1799560.6,5578803.5,13143956.6,593,235662.5,17849.3,321288.2,4830530.4,6599491.4,2035223.1
2,1200741010700100571330001,1,10,1712.0,580,126981.2,1440454.2,3175785.5,593,132755.9,22462.7,404329.0,-5792166.5,9112543.8,259737.1
3,1200741010700100571330001,2,10,1712.0,481,142722.5,794212.9,1620194.3,606,340591.4,37110.0,667980.3,-3494520.1,5299380.9,483313.8
4,1200741010700100571330001,3,10,2001.4,580,392671.5,2688525.5,6048071.2,593,0.0,0.0,0.0,766349.5,4889050.2,392671.5


In [163]:
#calculate the sum of all rows
#perform sum
stand.sum(axis=0)
#divide by acres for pa values

biosum_cond_id                   inf
rxpackage                        inf
owngrpcd                      9870.0
acres                      1609168.8
merch_haul_cost_psite       429137.0
merch_haul_cost_sum      436465474.7
merch_vol_cf_sum        3157098110.3
merch_dollars_val_sum   6867485310.5
chip_haul_cost_psite        466123.0
chip_haul_cost_sum       238252464.1
chip_yield_gt_sum         35857545.8
chip_dollars_val_sum     645435823.6
net_rev_dollars_sum     1087600823.1
harv_costs_sum          5752625226.7
haul_costs_sum           674717938.8
dtype: float64

In [164]:
#which stands had a higher max_nr by including bioenergy wood?
#add column bioRev
product['bioRev']=np.NaN
#if max nr is greater than merch nr, then bioRev is Y
product.loc[product.max_nr_dpa > product.merch_nr_dpa,['bioRev']] = 'Y'; product
#if max nr is the same as merch_nr then we know including chips made it less profitable, so bioRev is N
product.loc[product.max_nr_dpa == product.merch_nr_dpa,['bioRev']] = 'N'; product
product.head(n=5)

,biosum_cond_id,rxpackage,chip_yield_cf,merch_yield_cf,chip_yield_gt,merch_yield_gt,chip_val_dpa,merch_val_dpa,harvest_onsite_cpa,haul_chip_cpa,haul_merch_cpa,merch_chip_nr_dpa,merch_nr_dpa,max_nr_dpa,bioRev
0,1200641050705900795510001,15,366.9,755.0,8.5,17.6,153.7,1548.5,2367.2,20.9,44.7,-730.5,-863.3,-730.5,Y
1,1200741010700100522500001,1,425.9,3064.5,9.8,67.2,176.5,7220.0,3625.1,129.5,988.5,2653.4,2606.4,2653.4,Y
2,1200741010700100571330001,1,603.4,841.4,13.1,18.1,236.2,1855.0,5322.8,77.5,74.2,-3383.3,-3541.9,-3383.3,Y
3,1200741010700100571330001,2,603.4,841.4,13.1,18.1,236.2,1855.0,5322.8,77.5,74.2,-3383.3,-3541.9,-3383.3,Y
4,1200741010700100571330001,3,986.9,463.9,21.7,9.4,390.2,946.4,3095.4,198.9,83.4,-2041.2,-2232.4,-2041.2,Y


In [165]:
#count the number of stands that fall into the bioRev = Y and bioRev = N categories. 
p=product[['biosum_cond_id','bioRev']].groupby(['bioRev']).count().reset_index()
print(p)

  bioRev  biosum_cond_id
0      N              75
1      Y            1429


In [166]:
#what is the percentage?
per = (743/(40+743))*100
print(per)

94.89144316730524


# Effect of closing a mill on area treated and net revenue overall.

For this inquiry I created a new core analysis run, weightedAvg_mill, in which everything is the same as weightedAvg, but two mills were removed.  Elkhorn Biomass and Haines merch wood mill. The results show what happens when a regional mill closes. 

In [167]:
#read in the weighted_NR_mill table (which is the cycle1_best_rx_summary from CORE ANALYSIS for weightedAvg_mill scenario, not based on NET REV)
mill = pd.read_excel(open('data/weighted_NR_mill.xlsx', 'rb'), sheetname='cycle1_best_rx_summary', lowmemory=False)


In [168]:
#display top 5 lines of table
mill.head(n=5)

,biosum_cond_id,rx,acres,owngrpcd,optimization_value,tiebreaker_value,rx_intensity
0,1200741010700100571330001,107,1998.1,10,-0.3,84.7,nan
1,1200741010700100600620001,104,1820.5,10,-0.2,83.3,nan
2,1200741010700100736160001,115,2001.4,10,-0.0,23.7,nan
3,1200741010700100745730001,114,1764.1,10,-0.2,61.1,nan
4,1200741010700100814910001,116,1799.2,10,-0.5,85.9,nan


In [169]:
#group by owner and hazard score, display acres
groupby_w = mill[['owngrpcd', 'rx', 'acres']].groupby(['owngrpcd', 'rx'])


In [170]:
#calculate the sum of acres, by ownergroup and hazard score, for intial conditions, also provide a count. 
mill = groupby_w.agg('sum')\
                .rename(columns = lambda x: x + ' sum')\
                .join(pd.DataFrame(groupby_w.size(), 
                                      columns=['counts']))\
                .reset_index()

#print the first five lines of the new table
mill.head(n=5)

,owngrpcd,rx,acres sum,counts
0,10,100,22183.9,13
1,10,101,1978.8,2
2,10,102,18739.6,10
3,10,103,1885.6,1
4,10,104,156987.7,87


In [171]:
#find percentage of acres treated by each rx by owner group
#NOTE: percentages are of owner group acre totals, NOT of combined total acres
mill['percent']=np.NaN
mill['percent'][mill.owngrpcd==10]=(mill['acres sum']/fed)*100
mill['percent'][mill.owngrpcd==40]=(mill['acres sum']/private)*100
fe_nNR.head(n=5)

,owngrpcd,rx,acres sum,counts,percent
0,10,100,11674.0,9,0.5
1,10,101,5974.1,4,0.3
2,10,102,8918.1,6,0.4
3,10,103,13506.2,7,0.6
4,10,104,77155.4,42,3.6


Dataframe **fe_nNR**: ownergroup, rx, sum of acres, counts, and percent of subgroup total acres, that COULD be treated effectively by at least one treatment. 

In [172]:
#total up acres that are able to be treated by an rx by ownertype
fed_treat3=mill[mill["owngrpcd"]==10].sum()["acres sum"]
private_treat3=mill[mill["owngrpcd"]==40].sum()["acres sum"]

In [173]:
print(fed_treat3)
print(private_treat3)

986763.361601
317360.870539


Total acres that could be treated by any rx with a net revenue > 0:
* **Federal**: 986,763
* **Private**: 317,360

In [174]:
#total up percent of acres that are able to be treated by an rx by ownertype
fed_treat3P=mill[mill["owngrpcd"]==10].sum()["percent"]
private_treat3P=mill[mill["owngrpcd"]==40].sum()["percent"]

In [175]:
print(fed_treat3P)
print(private_treat3P)

45.8048987994
38.0662358147


**45% of federal acres** could be effectively treated by one or more of the treatment packages. 

**38% of private acres** could be effectively treated by one or more of the treatment packages.  

Which means that removing these mills from the analysis does not change the number of acres that can be effectively treated which a net revenue greater than 0.  Very interesting. 

# In that case, how does removing mills change net revenue values? How do harvest costs compare? Does this change the profitability of bioenergy? 


In [176]:
#read in the product cost/rev table from core analysis for rxpackage for the 'best' treatments (weighted average for mill scenario)
productM = pd.read_excel(open('data/millCompare.xlsx', 'rb'), sheetname='compare', lowmemory=False)

In [177]:
productM.head(n=5)
productM.to_csv('outputs/productM.csv')

In [178]:
#export to csv for R graphing
productMelt = pd.melt(productM, id_vars=['biosum_cond_id'], value_vars=['max_nr_dpa', 'mill_max_nr'])
productMelt.to_csv('outputs/productMelt.csv')

In [179]:
Image(url= "graphs/mill.png", width=400, height=400)

In [180]:
Image(url= "graphs/mill2.png", width=400, height=250)

In [182]:
ipython nbconvert --to pdf Explore BM Questions.ipynb

SyntaxError: invalid syntax (<ipython-input-182-e3632db641c1>, line 1)